In [ ]:

import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
import tarfile
import pandas as pd

In [ ]:
n_components = 6

(393, 393, 64)

In [3]:
#Load wild-type disto

WT = np.load('prova/embeddings_WT.npz')
WT_pre_disto = WT['disto']
WT_disto = WT_pre_disto.squeeze()

dist = np.linspace(2, 22, 64)
wt_mode_matrix = dist[np.argmax(WT_disto, axis=2)]

#linearize matrix
wt_vector = wt_mode_matrix[np.triu_indices(wt_mode_matrix.shape[0], k=1)]

wt_vector.shape

(77028,)

In [22]:
def linearize_disto_npz(file):
    try:
        variant = np.load(file)       
    except:
        return None    
    pre_disto = variant['disto']

    disto = pre_disto.squeeze()

    #Create a list of labels (the distances bins)
    labels = np.linspace(2, 22, 64)

    #Extract the most probable distance inferred by Boltz
    mode_matrix = labels[np.argmax(disto, axis=2)]

    #Linearize using the upper triangular matrix
    linear_upper = mode_matrix[np.triu_indices(mode_matrix.shape[0], k=1)]

    linear_upper.shape

    return(linear_upper)

In [24]:
error = []
vector = []
var = []

with tarfile.open('prova/dummy.tar', 'r') as tar:
    for member in tar :
        with tar.extractfile(member) as f:
            mut_vector = linearize_disto_npz(f)
            if mut_vector is None:
                error.append(member.name)
                continue
            mut_name = Path(member.name).stem.split('_')[1]  # estrae variante

            vect_diff = wt_vector - mut_vector
            vector.append(vect_diff)
            var.append(mut_name)

In [ ]:

print(len(vector) , var)


6 ['A119C', 'A119D', 'A119E', 'A119F', 'A119G', 'WT']


In [ ]:
#Stack list into matrix
X = np.vstack(vector)

#Remove all-zero columns
v_difference_matrix = X[:, ~(X == 0).all(0)]

#Apply PCA to X
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

#Save PCA embeddings to CSV
df1 = pd.DataFrame(X_pca, columns=[i for i in range(n_components)])
df1.insert(0,"", var)
df1.to_csv("pca_dummy.csv", index=False)

#Save vector differences embedding to CSV
df2 = pd.DataFrame(v_difference_matrix, columns=[i for i in range(v_difference_matrix.shape[1])])
df2.insert(0,"", var)
df2.to_csv("vector_difference_dummy.csv", index=False)